In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph()
graph

In [5]:
movie_query = """
LOAD CSV WITH HEADERS FROM 
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') |
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') |
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

In [6]:
movie_query

"\nLOAD CSV WITH HEADERS FROM \n'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row\n\nMERGE(m:Movie{id:row.movieId})\nSET m.released = date(row.released),\n    m.title = row.title,\n    m.imdbRating = toFloat(row.imdbRating)\nFOREACH (director in split(row.director, '|') |\n    MERGE (p:Person {name:trim(director)})\n    MERGE (p)-[:DIRECTED]->(m))\nFOREACH (actor in split(row.actors, '|') |\n    MERGE (p:Person {name:trim(actor)})\n    MERGE (p)-[:ACTED_IN]->(m))\nFOREACH (genre in split(row.genres, '|') |\n    MERGE (g:Genre {name:trim(genre)})\n    MERGE (m)-[:IN_GENRE]->(g))\n"

In [7]:
graph.query(movie_query)

[]

In [8]:
graph.refresh_schema()

In [10]:
print(graph.schema)

Node properties:
Person {name: STRING}
Movie {id: STRING, title: STRING, released: DATE, imdbRating: FLOAT}
User {name: STRING, city: STRING, userId: INTEGER, age: INTEGER}
Posts {userId: INTEGER, postId: INTEGER, content: STRING, timestamp: DATE_TIME}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)
(:Movie)-[:IN_GENRE]->(:Genre)
(:User)-[:POSTED]->(:Posts)
(:User)-[:FRIEND]->(:User)
(:User)-[:LIKES]->(:User)
